In [1]:
from pyspark.ml.fpm import FPGrowth
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_set, desc

In [2]:
spark = SparkSession.builder.appName('main').getOrCreate()

22/09/08 17:45:24 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.100.238 instead (on interface wlp4s0)
22/09/08 17:45:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/08 17:45:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/08 17:45:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# spark.conf.set('spark.driver.memory', '8g')
# spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', 'true')

In [4]:
dataset_path = 'inputs/by-manufacturer.csv'
output_path = 'outputs/by-manufacturer'

In [5]:
def check(df):
    print(f"Count: {df.count()}")
    print('\nElements:')
    print(df.show(5))
    # print(df.show(5, truncate = False))

In [6]:
df = spark.read.options(header = True).csv(dataset_path)

In [7]:
# check(df)

In [8]:
df = df.groupBy('sale_code').agg(collect_set('barcode'))

In [9]:
df = df.withColumnRenamed('collect_set(barcode)', 'items')

In [10]:
# check(df)

In [11]:
fp_growth = FPGrowth(itemsCol = 'items', minSupport = 0.0005, minConfidence = 0.001)

In [12]:
model = fp_growth.fit(df)

In [13]:
rules = model.associationRules

In [14]:
rules = rules.sort(col('lift').desc())

In [15]:
rules = rules[rules['lift'] >= 1]

In [16]:
rules.toPandas().to_csv(f"{output_path}/association_rules.csv", index = False)

22/09/08 17:50:31 ERROR Executor: Exception in task 0.0 in stage 25.0 (TID 12)
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.util.Arrays.copyOf(Arrays.java:3745)
	at java.base/java.io.ByteArrayOutputStream.grow(ByteArrayOutputStream.java:120)
	at java.base/java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:95)
	at java.base/java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:156)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.write(ObjectOutputStream.java:1849)
	at java.base/java.io.ObjectOutputStream.write(ObjectOutputStream.java:708)
	at org.apache.spark.util.Utils$.writeByteBuffer(Utils.scala:244)
	at org.apache.spark.scheduler.DirectTaskResult.$anonfun$writeExternal$1(TaskResult.scala:53)
	at org.apache.spark.scheduler.DirectTaskResult$$Lambda$2679/0x0000000841168c40.apply$mcV$sp(Unknown Source)
	at scala.runtime.java8.JFunction0$m

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/cfascina/Projetos/frequent-pattern-analysis/.env/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/cfascina/Projetos/frequent-pattern-analysis/.env/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/cfascina/Projetos/frequent-pattern-analysis/.env/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/cfascina/Projetos/frequent-pattern

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
# freq_itemsets = model.freqItemsets

In [ ]:
# freq_itemsets = freq_itemsets.select('freq', 'items')

In [ ]:
# freq_itemsets = freq_itemsets.sort(col('freq').desc())

In [ ]:
# check(freq_itemsets)

In [ ]:
# freq_itemsets3.toPandas().to_csv(f"{output_path}/frequent_itemsets.csv", index = False)

In [ ]:
# transformed = model.transform(df)

In [ ]:
# check(transformed)

In [ ]:
# transformed.toPandas().to_csv(f"{output_path}/transformed.csv", index = False)